In [ ]:
##MODEL 2.6

In [1]:
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer

torch.manual_seed(0)

model = AutoModel.from_pretrained('openbmb/MiniCPM-V-2_6', trust_remote_code=True)
model = model.to(device='cuda')
tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-V-2_6', trust_remote_code=True)
model.eval()

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-09-03 20:11:34,353] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.0
 [WARNING]  using untested triton version (2.0.0), only 1.0.0 is known to be compatible


Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:08<00:00,  2.01s/it]


MiniCPMV(
  (llm): Qwen2ForCausalLM(
    (model): Qwen2Model(
      (embed_tokens): Embedding(151666, 3584)
      (layers): ModuleList(
        (0-27): 28 x Qwen2DecoderLayer(
          (self_attn): Qwen2Attention(
            (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
            (k_proj): Linear(in_features=3584, out_features=512, bias=True)
            (v_proj): Linear(in_features=3584, out_features=512, bias=True)
            (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
            (rotary_emb): Qwen2RotaryEmbedding()
          )
          (mlp): Qwen2MLP(
            (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
            (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
            (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
          (post_attention_layernorm): Qwen2

In [4]:
image = Image.open('/home/models/output__lora/2024-06-12___13-06-24.jpg').convert('RGB')

# First round chat 
question = 'This is an image of the inside of a bin. Please list all the objects that you can detect.'
msgs = [{'role': 'user', 'content': question}]

answer = model.chat(
    image=image,
    msgs=msgs,
    context=None,
    tokenizer=tokenizer,
    sampling=True
)

print(answer)

The objects that can be detected in the image are:

1. A blue plastic bin liner, which is partially visible at the bottom of the frame.
2. Several pieces of white paper or napkins; some appear to be crumpled and others are flat.
3. The contents of a black trash bag, which is also visible within the bin.

These items suggest that the bin contains discarded waste materials such as used paper products and possibly other small refuse items.


In [ ]:
##Model 2.6 Fine tuning

In [2]:
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer
from peft import PeftModel

path_to_adapter = "/home/models/output/output__lora"
torch.manual_seed(0)

model = AutoModel.from_pretrained('openbmb/MiniCPM-V-2_6', trust_remote_code=True)
lora_model = PeftModel.from_pretrained(
    model,
    path_to_adapter,
    device_map="auto",
    trust_remote_code=True
)

lora_model = lora_model.to(device='cuda')
tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-V-2_6', trust_remote_code=True)
lora_model.eval()

Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:14<00:00,  3.62s/it]


PeftModel(
  (base_model): LoraModel(
    (model): MiniCPMV(
      (llm): Qwen2ForCausalLM(
        (model): Qwen2Model(
          (embed_tokens): ModulesToSaveWrapper(
            (original_module): Embedding(151666, 3584)
            (modules_to_save): ModuleDict(
              (default): Embedding(151666, 3584)
            )
          )
          (layers): ModuleList(
            (0-27): 28 x Qwen2DecoderLayer(
              (self_attn): Qwen2Attention(
                (q_proj): lora.Linear(
                  (base_layer): Linear(in_features=3584, out_features=3584, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=3584, out_features=64, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=64, out_features=3584, bias=False)
               

In [23]:
image = Image.open('/home/models/output__lora/4239dbcf126d129f_02_09_2024__18_03_42.jpg').convert('RGB')
question = 'You are an expert in recognizing objects. The objects are showned to the camera by persons. Which object is showned to the camera?. Please answer withi ine word'
msgs = [{'role': 'user', 'content': question}]

answer = lora_model.chat(
    image=image,
    msgs=msgs,
    context=None,
    tokenizer=tokenizer,
    sampling=True
)
print(answer)

PLASTIC BAG


In [ ]:
##Model 2.5

In [2]:
# test.py  Need more than 16GB memory.
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained('openbmb/MiniCPM-Llama3-V-2_5', trust_remote_code=True, low_cpu_mem_usage=True)
model = model.to(device='cuda')

tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-Llama3-V-2_5', trust_remote_code=True)
model.eval()


Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:09<00:00,  1.42s/it]
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/image_processing_auto.py:513: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


ValueError: too many values to unpack (expected 3)

In [11]:
image = Image.open('/home/models/output__lora/33912d60fa7415f8_02_09_2024__13_30_28.jpg').convert('RGB')
question = 'Which object is shown to the camera? Answer the question using a single word or phrase.'
msgs = [{'role': 'user', 'content': question}]

answer = model.chat(
    image=image,
    msgs=msgs,
    context=None,
    tokenizer=tokenizer,
    sampling=True
)
print(answer)

key fob
